#  Assignment 5

## Notebook made by  

|** Name** | **Student id** | **email**|
|:- |:-|:-|
|. | | |
|  | |. |

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. 

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='link to your selfie'/>

### Note
* **Assignments without the selfies or completely filled in information will not be graded and receive 0 points.**


We maken de toetsassignment af. We bekijken dus weer het artikel <http://language.media.mit.edu/paper>. In de toets hebben we de netwerkdata gebaseerd op boekvertalingen gemaakt.
In dit huiswerk gaan we verder.

 

In [1]:
%matplotlib inline
import pandas as pd
import re
from lxml import etree 
from bz2file import BZ2File
import codecs
import nltk
from collections import defaultdict
from  itertools import combinations  # ideal for creating all possible pairs that out can make out of a set
from __future__ import division
from math import sqrt

### Deel  Wikipedia  

 

1. Maak   van de Wikipedia _raw_ dataset de netwerk  datasets die op <http://language.media.mit.edu/data> beschreven staan. 
2. We bekijken in hoeverre onze resultaten afwijken van die in de geleverde bewerkte datasets.
3. We leggen eventuele verschillen uit.

#### In Stapjes

We doen de opdracht in een aantal stapjes.

#### Let op, je notebook moet vanzelf draaien. Alle data moet opgehaald worden.

# Download 

1. Download de ruwe dataset als zip file, ontzip hem en sla hem in de directory op waarin dit notebook ook staat.
    * Dit mag je met de hand doen. De code **moet niet runbaar** in je notebook staan.
    * Maar zorg dat de rest van de code er vanuit gaat dat `wikipedia_userlang_iso639-3.tsv` in dezelfde folder staat als dit notebook
2. Lees de 2 bewerkte datasets over Wikipedia _Wikipedia Stats by Language_ en _Edges_ direct in van de urls in  pandas Dataframes, die je respectivelijk `dataset_stats_wikipedia` en `wikipedia_edges` noemt.
    1. Maak de Taal code de index in het eerste frame, en hou alleen de Kolommen `['Editors', 'Edits','Multilinguals','Edits by mulitlinguals']`
    2. Sorteer het frame op de index
    2. Verwijder de kolommen `SourceLanguageName	TargetLanguageName	SourcePopulation	TargetPopulations` uit het tweede frame
    3. Sorteer het frame op `SourceLanguageName`
    3. Laat steeds met `head()` zien dat het werkt.
    
### Je antwoord bestaat uit 2 cellen

In [31]:
#1
#wikipedia_userlang = pd.read_csv('wikipedia_userlang_iso639-3.tsv', sep = '\t')

#2
dataset_stats_wikipedia = pd.read_csv('http://language.media.mit.edu/data/public/dataset_stats_wikipedia.tsv', sep = '\t')
wikipedia_edges =  pd.read_csv('http://language.media.mit.edu/data/public/books_edges.tsv', sep = '\t')

#2A & 2B
dataset_stats_wikipedia = pd.pivot_table(dataset_stats_wikipedia,index=['Code'] ,values=['Editors', 'Edits','Multilinguals','Edits by mulitlinguals']).sort_index()

dataset_stats_wikipedia.head(20)

,Editors,Edits,Edits by mulitlinguals,Multilinguals
Code,,,,
aar,4,46,9,3
abk,15,2807,1494,7
afr,511,103652,95988,352
aka,31,327,240,24
amh,68,4596,4039,45
ang,86,9915,9504,64
ara,18258,2178719,655776,4647
arc,21,2152,2049,13
arg,141,60924,58810,103


In [30]:
#2C & 2D
wikipedia_edges2 = wikipedia_edges.drop(['SourceLanguageName', 'TargetLanguageName', 'SourcePopulation', 'TargetPopulations'], axis=1).sort_values(by='SourceLanguageCode', ascending=True)

#2E
wikipedia_edges2.head()

,SourceLanguageCode,TargetLanguageCode,Coocurrences,PhiCorrelation,Tstatistic
0,afr,eng,271,0.013648,17.207148
1,afr,nld,78,0.015377,3.332197
2,amh,eng,28,0.004344,5.476711
3,ang,eng,51,0.006363,8.021067
4,ara,deu,446,0.007497,3.550224


# Haal gegevens uit de ruwe data, deel 1

* De file `'wikipedia_userlang_iso639-3.tsv'` lijkt een spreadsheet maar hij heeft niet op elke regel evenveel kolommen.
* Processing met pandas is dus ook lastig.
* Als alternatief gaan we file zelf regel voor regel  inlezen, en de gewenste gegevens opslaan.
* Ons einddoel is om een dataframe te maken dat er precies zo uitziet als `dataset_stats_wikipedia`
* We doen dat in 2 stappen:
    1. We maken een aantal dicts aan met de gewenste gegevens
    2. Van elke dict maken we een dataframe met `pd.DataFrame.from_dict`
    3. Die plakken we allemaal aan elkaar met `join` (neem een inner join)
    4. Noem het eindresultaat `stats`, sorteer het ook op de index en laat het begin zien met `head`.
    5. Check natuurlijk zelf of je op de goede weg zit door je resultaat te vergelijken met `dataset_stats_wikipedia`
* Hieronder staat wat code om je op weg te helpen
* **Lees het artikel en de supplementary material goed om net als zij de ruwe data te filteren.**


# Haal gegevens uit de ruwe data, deel 2

* In dezelfde code waarin je de vorige vraag hebt gemaakt, maak je ook een dict `pairs` aan. Hier zijn de sleutels paren van taalcodes, **waarbij de eerste altijd alfabetisch eerder is dan de tweede**, en hun waarde het aantal cooccurences.
* Maak van dit dict een Dataframe `edges`, waarbij `edges.columns=['SourceLanguageCode','TargetLanguageCode','Coocurrencesmy']`
* Sorteer ook hier op `'SourceLanguageCode'`
* **Lees het artikel en de supplementary material goed om net als zij de ruwe data te filteren.**

In [4]:
with open('wikipedia_userlang_iso639-3.tsv') as f:    
    c=0
    editors=defaultdict(int)
    edits=defaultdict(int)
    multilingual=defaultdict(int)
    Edits_by_multilinguals=defaultdict(int)
    pairs= defaultdict(int)
    for l in f:
        c += 1
        totaled = 0
        line = l.split('\t')[1:]
        cells = []
        for cell in line:
            cell = cell.split(',')
            if len(cell) == 2:
                totaled += int(cell[1])
                cells.append(cell)
        if totaled >= 5 and len(cells) <= 5:
            for cell in line:
                cell = cell.split(',')
                if len(cell) == 2 and cell[1]>=2:
                    editors[cell[0]] += 1
                    edits[cell[0]] += int(cell[1])
            #Add to the multilingual dict and edit by multilingual
            if len(cells) > 1:
                combi = []
                for x in cells:
                    combi.append(x[0])                
                    multilingual[x[0]] += 1
                    Edits_by_multilinguals[x[0]] += int(x[1])
                combi = list(combinations(sorted(combi), 2))
                for pair in combi:
                    if pair[0]<pair[1]:
                         pairs[pair] += 1

#        if c == 2000:
#            print pairs
#            break

In [156]:
#Deel 1
#Aanmaken Dataframe editors
editorsdf = pd.DataFrame.from_dict(editors, orient='index')
editorsdf.columns = ['Editors']

#Aanmaken Dataframe edits
editsdf = pd.DataFrame.from_dict(edits, orient='index')
editsdf.columns = ['Edits']

#Aanmaken Dataframe multilingual
multilingualdf = pd.DataFrame.from_dict(multilingual, orient='index')
multilingualdf.columns = ['Multilinguals']

#Aanmaken Dataframe Edits_by_mulitlinguals
Edits_by_multilingualsdf = pd.DataFrame.from_dict(Edits_by_multilinguals, orient='index')
Edits_by_multilingualsdf.columns = ['Edits by mulitlinguals']

stats = pd.concat([editorsdf, editsdf, multilingualdf, Edits_by_multilingualsdf], join='inner',axis=1)
stats.sort_index().head()

,Editors,Edits,Multilinguals,Edits by mulitlinguals
aar,8,51,8,51
abk,23,2815,17,1540
afr,730,103893,600,97942
aka,49,346,43,279
amh,88,4619,65,4062


In [6]:
#Deel2
#Aanmaken Dataframe Edges
edges = pd.DataFrame(pairs.values(), index=pd.MultiIndex.from_tuples(pairs.keys())).sort_index().reset_index()
edges.columns=['SourceLanguageCode','TargetLanguageCode','Coocurrencesmy']
edges.head(10)

,SourceLanguageCode,TargetLanguageCode,Coocurrencesmy
0,aar,arg,1
1,aar,bis,1
2,aar,deu,3
3,aar,eng,2
4,aar,fin,1
5,aar,ipk,1
6,aar,ita,3
7,aar,kat,1
8,aar,kor,1
9,aar,lao,1


# Vergelijk 

We gaan nu onze dataframes vergelijken met die we hebben opgehaald.

We zijn natuurlijk erg benieuwd of er verschillen zijn, en vooral ook groot die dan zijn, als ze er zijn.

1. Join `dataset_stats_wikipedia` en `stats` in een nieuwe dataframe `samen`. 
    * laat met `samen.describe()` zien hoe groot of klein de verschillen zijn
2. Maak een functie `fout_percentage` waar je twee kolommen en een dataframe aan geeft en die _het verschil tussen die twee kolommen bepaalt, en omzet naar een "foutpercentage"_.
3. Bereken dat percentage voor de vier kolommen `['Editors', 'Edits','Multilinguals','Edits by mulitlinguals']` en print het netjes uit.
4. Leg in een paar zinnen uit waarom jouw maat voor "foutpercentage" echt ergens op slaat, en het een goede indicator geeft. 
5. Doe tenslotte hetzelfde voor `edges`. 
    * Hier doe je natuurlijk een `merge`
    * Bekijk hier eens het verschil tussen een outer en een inner join.
    * Leg uit wat er hier aan de hand is.
    * Kan je het verschil verklaren?


In [246]:
#1 Join dataset_stats_wikipedia en stats in een nieuwe dataframe samen
stats2 = stats.copy(deep=True)
stats2.columns = ['Editors_my', 'Edits_my','Multilinguals_my','Edits by mulitlinguals_my']
samen= pd.concat([dataset_stats_wikipedia, stats2], axis=1, join='outer')
# vergelijk stats met dataset_stats_wikipedia
samen.describe()

,Editors,Edits,Edits by mulitlinguals,Multilinguals,Editors_my,Edits_my,Multilinguals_my,Edits by mulitlinguals_my
count,238.000000,2.380000e+02,238.000000,238.000000,240.000000,2.400000e+02,240.000000,240.000000
mean,12243.554622,1.608757e+06,729135.878151,2648.226891,12927.566667,1.596299e+06,3762.166667,788132.262500
std,105148.317861,1.322120e+07,5342618.270859,15606.769464,108654.604906,1.317142e+07,21592.225214,5814437.662472
min,1.000000,2.000000e+00,2.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000
25%,20.250000,1.410500e+03,562.500000,13.000000,27.000000,1.399500e+03,19.000000,622.250000
50%,61.500000,8.092500e+03,5325.500000,39.000000,79.500000,7.748500e+03,55.000000,5412.500000
75%,458.250000,1.099002e+05,64802.000000,238.500000,538.250000,1.059285e+05,337.750000,61607.750000
max,1589250.000000,1.983610e+08,78341211.000000,218320.000000,1645182.000000,1.984296e+08,300326.000000,85598573.000000


In [250]:
#2 Maak een functie fout_percentage waar je twee kolommen en een dataframe aan geeft
# en die het verschil tussen die twee kolommen bepaalt, en omzet naar een "foutpercentage"

def fout_percentage(col1,col2,df):
    foutpercentage = ((df[col2].mean() - df[col1].mean()) / df[col1].mean()) * 100

    return foutpercentage

#3 Bereken dat percentage voor de vier kolommen ['Editors', 'Edits','Multilinguals','Edits by mulitlinguals'] en print het netjes uit.

for col in stats.columns:
#    print "%s\t%s\t%s" % (col, fout_percentage(col, col+'_my', samen))
    print col, fout_percentage(col, col+'_my', samen), "%"

Editors 5.5867112611 %
Edits -0.774409738568 %
Multilinguals 42.0636079106 %
Edits by mulitlinguals 8.09127435867 %


#### 4 Leg in een paar zinnen uit waarom jouw maat voor "foutpercentage" echt ergens op slaat, en het een goede indicator geeft.
## Tekst hier blablabblabal, moet nog ingevuld worden!!

In [ ]:
# vergelijk edges met "inner join"
 

In [ ]:
# vergelijk edges met "outer join"

edges2=pd.merge(edges,wikipedia_edges, how='outer', on=['SourceLanguageCode','TargetLanguageCode'])
print fout_percentage('Coocurrencesmy','Coocurrences', edges2)
edges2.describe()

# Compute our own PhiCorrelation and T-statistic

Maak een functie `PhiCorrelation(i,j)` die gegeven 2 talen, de $\phi$ correlatie en de $t$-statistic uitrekent volgens vergelijkingen [1] en [2] uit het artikel.

* Let vooral op hoe je $N$ bepaalt. En beschrijf goed waarom je precies die keuze hebt genomen. Je uitkomsten hangen daar sterk vanaf.

In [ ]:
# Compute our own PhiCorrelation and T-statistic


def PhiCorrelation(i,j):
     #Your code here
    return Mij,Mi, Mj,  N, PhiCorrelation, Tstatistic

# test
PhiCorrelation('afr','nld')

# Reflectie
Je hebt getracht een deel van het artikel te reproduceren. Ook hebben we gedacht over het artikel en de mate van herhaalbaarheid van de vermelde gegevens. 

Geef nu in een half A4 je eigen mening. Behandel deze twee punten:

1. Wat was al goed gedaan?
2. Wat kon beter en hoe dan?